# RAG System

The goal of this notebook is to develop a streamlined tool that will parse in a natural language request or receive a json object to retrieve specific records.

For example a user may request to the assistant "What are the ten streets with most pothole's?"

The RAG system is only one piece of the puzzle to an agentic virtual assistant that will allow executives to make informed decisions such as budgeting, project prioritization, evaluation of success, among many other applications.

## Step 1) Develop retrieval tool

We will start off with the development of the retrieval tool

In [75]:
import pandas as pd # this allow us to read in data and perform eda
import numpy as np # used to manipulate and transform data efficiently
import duckdb # will serve as our local database
import re
from typing import List, Optional, Union

Our RAG solution should receive either street name, district, zipcode, or a combination of such. We may also send in optional parameters such as year, month, day (Monday, Tuesday,...) for which to base the query on, and return the relevant records.

The current implementation ---...

In [76]:
conn = duckdb.connect('potholess.db') # establish connection to local database
conn.sql("""
    CREATE TABLE IF NOT EXISTS potholes
    AS SELECT * FROM 'potholes.parquet'
""") # creates a table if it does not already exist, and loads in the data stored in the parquet file

In [77]:
import re
from typing import Optional, Union, List

def query_table(street=None, year=None, zipcode=None, district=None):
    base_query = """
        SELECT latitude, longitude, street_name, year, council_district
        FROM potholes WHERE 1=1
    """
    params = []

    if isinstance(street, str):
        safe_street = street.replace("'", "''")
        base_query += f" AND street_name ILIKE '%{safe_street}%'"

    if isinstance(year, int):
        base_query += " AND year = ?"
        params.append(year)
    elif year in ("historical", None):
        pass  # no year filter
    else:
        raise ValueError("Year must be an integer, 'historical', or None")

    if isinstance(zipcode, int):
        base_query += " AND zipcode = ?"
        params.append(zipcode)

    if isinstance(district, int):
        base_query += " AND council_district = ?"
        params.append(district)

    # Place the debug prints here, after base_query is fully constructed
    print("QUERY:", base_query)
    print("PARAMS:", params)

    return conn.sql(base_query, params=params).fetchall()
# def query_table(street=None, year: Union[int, str, None] = 2024, zipcode = None, district = None) -> List[tuple]:
#     """
#     Return pothole records where street name matches a full word (case-insensitive),
#     optionally filtered by year (2018–2024), or include all years with 'historical' or None.

#     Args:
#         street: Target street name (matched as a full word).
#         year: Integer year (2018–2024), or 'historical'/None for all years.
#     """
    

#     base_query = """
#         SELECT latitude, longitude, street_name, year, council_district
#         FROM potholes WHERE 1=1"""
#     params = []

#     if isinstance(street, str):
#         safe_pattern = r"\b" + re.escape(street) + r"\b"
#         base_query += " AND REGEXP_MATCHES(street_name, ?, 'i')"
#         params.append(safe_pattern)

#     if isinstance(year, int) and 2018 <= year <= 2024:
#         base_query += " AND year = ?"
#         print(f'specified year: {year}')
#         params.append(year)
#     elif year in ("historical", None):
#         print('no year filter')
#         pass  # no year filter
#     else:
#         raise ValueError("Year must be 2018–2024, 'historical', or None")

#     if isinstance(zipcode, int):
#         base_query += " AND zipcode = ?"
#         print(f'specified zip: {zipcode}')
#         params.append(zipcode)
#     else:
#         print('Searching all zips')

#     if isinstance(district, int):
#         base_query += " AND council_district = ?"
#         print(f'specified district: {district}')
#         params.append(district)
#     else:
#         print('Searching all district')
#     print(base_query)
#     print(params)

#     return conn.sql(base_query, params=params).fetchall()

In [78]:
query_table('Main', zipcode=78204) # case for which only street and zipcode are provided, year is defaulted to 2024, district not specified

QUERY: 
        SELECT latitude, longitude, street_name, year, council_district
        FROM potholes WHERE 1=1
     AND street_name ILIKE '%Main%' AND zipcode = ?
PARAMS: [78204]


[(29.4152561051478, -98.4966523120534, 'S MAIN AVE and W SHERIDAN', 2024, 1),
 (29.4152561051478, -98.4966523120534, 'S MAIN AVE and W SHERIDAN', 2024, 1),
 (29.4218619032057, -98.4943894335149, 'S MAIN AVE', 2024, 1),
 (29.4148036832604, -98.4970157737168, 'MAIN AV S', 2018, 1),
 (29.4109451695356, -98.4985016105287, 'S MAIN AVE', 2020, 1),
 (29.4109451695356, -98.4985016105287, 'S MAIN AVE', 2022, 1)]

In [79]:
query_table('Main', year=2020, zipcode=78205) # case for which street, year, and zipcode are provided

QUERY: 
        SELECT latitude, longitude, street_name, year, council_district
        FROM potholes WHERE 1=1
     AND street_name ILIKE '%Main%' AND year = ? AND zipcode = ?
PARAMS: [2020, 78205]


[(29.4293463207256, -98.4939283493591, 'E MARTIN ST and N MAIN AVE', 2020, 1),
 (29.4285761721465, -98.4939070232682, 'E PECAN ST and N MAIN AVE', 2020, 1),
 (29.4293463207256, -98.4939283493591, 'E MARTIN ST and N MAIN AVE', 2020, 1),
 (29.4331144968272, -98.4940196838845, 'BUFFALO RUN and N MAIN AVE', 2020, 1),
 (29.4285761721465, -98.4939070232682, 'E PECAN ST and N MAIN AVE', 2020, 1),
 (29.4293463207256, -98.4939283493591, 'E MARTIN ST and N MAIN AVE', 2020, 1),
 (29.4287670299109, -98.4941951093899, 'N MAIN AVE', 2020, 1)]

In [80]:
query_table('Main', year='historical', zipcode=78204) # case for which street, year, and zipcode are provided, year is historical

QUERY: 
        SELECT latitude, longitude, street_name, year, council_district
        FROM potholes WHERE 1=1
     AND street_name ILIKE '%Main%' AND zipcode = ?
PARAMS: [78204]


[(29.4152561051478, -98.4966523120534, 'S MAIN AVE and W SHERIDAN', 2024, 1),
 (29.4152561051478, -98.4966523120534, 'S MAIN AVE and W SHERIDAN', 2024, 1),
 (29.4218619032057, -98.4943894335149, 'S MAIN AVE', 2024, 1),
 (29.4148036832604, -98.4970157737168, 'MAIN AV S', 2018, 1),
 (29.4109451695356, -98.4985016105287, 'S MAIN AVE', 2020, 1),
 (29.4109451695356, -98.4985016105287, 'S MAIN AVE', 2022, 1)]

In [81]:
candidates = query_table(street='San Pedro', year=2021, zipcode=78212, district=1)

QUERY: 
        SELECT latitude, longitude, street_name, year, council_district
        FROM potholes WHERE 1=1
     AND street_name ILIKE '%San Pedro%' AND year = ? AND zipcode = ? AND council_district = ?
PARAMS: [2021, 78212, 1]


In [82]:
len(candidates)

2

In [83]:
candidates

[(29.4885498296817, -98.4988725376364, 'SAN PEDRO AVE', 2021, 1),
 (29.4890345103144, -98.4997786282976, 'SAN PEDRO AVE', 2021, 1)]

In [84]:
candidates = query_table(street='San Pedro', year=2021, zipcode=78216, district=1)

QUERY: 
        SELECT latitude, longitude, street_name, year, council_district
        FROM potholes WHERE 1=1
     AND street_name ILIKE '%San Pedro%' AND year = ? AND zipcode = ? AND council_district = ?
PARAMS: [2021, 78216, 1]


In [85]:
len(candidates)

29

## 

In [86]:
len(query_table(zipcode=78249))

QUERY: 
        SELECT latitude, longitude, street_name, year, council_district
        FROM potholes WHERE 1=1
     AND zipcode = ?
PARAMS: [78249]


987

: 

# Embeddings
We will now create embeddings for the distinct street entries. This will allow us to process requests such as 